In [53]:
import os
from operator import itemgetter
from typing import Dict, List, Optional, Sequence

import weaviate
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from ingest import get_embeddings_model
from langchain_anthropic import ChatAnthropic
from langchain_community.chat_models import ChatCohere
from langchain_community.vectorstores import Weaviate
from langchain_core.documents import Document
from langchain_core.language_models import LanguageModelLike
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import (
    ConfigurableField,
    Runnable,
    RunnableBranch,
    RunnableLambda,
    RunnablePassthrough,
    RunnableSequence,
    chain,
)

from langchain.indexes import SQLRecordManager, index

from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv



In [54]:
from langserve import RemoteRunnable

chain = RemoteRunnable("http://localhost:8080/chat/")

In [55]:
class ChatRequest(BaseModel):
    question: str
    chat_history: Optional[List[Dict[str, str]]]

chat_request = ChatRequest(question="What is streamlit?", chat_history=[{}])


In [ ]:
chain.invoke(chat_request)

In [57]:
async for chunk in chain.astream(chat_request):
    print(chunk, end="", flush=True)

INFO:httpx:HTTP Request: POST http://localhost:8080/chat/stream "HTTP/1.1 200 OK"


CancelledError: 

In [ ]:
chatHistory = [
  {
    "human": 'Hello, AI!',
    "ai": 'Hello, human! How can I assist you today?',
  },
  {
    "human": 'What is streamlit?',
    'ai': 'Streamlit is an open-source machine learning framework for building and sharing interactive, web-based data applications [$1$]. It allows users to build applications using Python, making it accessible to a wide range of developers [$2$]. With Streamlit, you can create real-time data visualizations, run machine learning models in the browser, and add user interfaces for model predictions [$3$]. The framework also supports both local and remote development environments [$4$]',
  },
  
]

In [ ]:
class ChatRequest(BaseModel):
    question: str
    chat_history: Optional[List[Dict[str, str]]]

chat_request = ChatRequest(question="Can I use it to implement model?", chat_history=chatHistory)

In [ ]:
ai = ''
async for chunk in chain.astream(chat_request):
    ai += chunk
    print(chunk, end="", flush=True)

INFO:httpx:HTTP Request: POST http://localhost:8080/chat/stream "HTTP/1.1 200 OK"


 Yes, Streamlit allows you to implement machine learning models within interactive web applications [$3$]. This feature enables real-time data visualizations and user interfaces for model predictions [$3$]. It supports both local and remote development environments for flexibility in building your application [$4$].

In [ ]:

history = [{}]
def user(mess, history):
    return history + [{"human": mess, "ai": None}]

def chat(history):
    for i in "xin chao":
        history[-1]["ai"] = i
        yield history


user("xin chao", history)
print(history)
chat(history)
print(history)

[{}]
[{}]


In [ ]:
history = [[]]
def user(mess, history):
    return history + [[mess,None]]

def chat(history):
    for i in "xin chao":
        history[-1][1] = i
        yield history


history = user("xin chao", history)
print(history)
chat(history)
print(history)

[[], ['xin chao', None]]
[[], ['xin chao', None]]


In [ ]:
a = []
def plus(a):
    return a + [3]

a = plus (a)